# Script to automatically make diacritical corrections in Hebrew text

## Defines letters and words

In [1]:
aleph_bet = ['א','ב','ג','ד','ה','ו','ז','ח','ט','י','כ','ך','ל','מ','ם',
             'נ','ן','ס','ע','פ','ף','צ','ק','ר','ש','ת','װ','ױ','ײ','יִ',
             'ﬡ','ﬢ','ﬣ','ﬤ','ﬥ','ﬦ','ﬧ','ﬨ','שׁ','שׂ','שּׁ','שּׂ','אַ','אָ',
             'גּ','דּ','הּ','וּ','זּ','טּ','יּ','ךּ','כּ','לּ','מּ','נּ','סּ','ףּ',
             'פּ','צּ','שּ','תּ','וֹ','בֿ','כֿ','פֿ','ﭏ','בּ', 'קּ']

cant = ['֑','֒','֓','֔','֕','֖','֗','֘','֙','֚','֛','֜',
             '֝','֞','֠','֡','֢','֣','֤','֥','֦','֧','֨','֩','֪','֫','֬','֭','֯','׃']

cant2 = cant[:28]

vowel = ['ְ','ֱ','ֲ','ֳ','ִ','ֵ','ֶ','ַ','ָ','ֹ','ֺ','ֻ','ּ','ֽ','־','ֿ','ׁ','ׂ','ׄ','ׅ','ׇ']

letter_with_dagesh = ['שּׁ','שּׂ','גּ','דּ','זּ','טּ','יּ','כּ','לּ','מּ','נּ','סּ','פּ','צּ','שּ','תּ','בּ','קּ']
rafe = 'ֿ'
shva = vowel[0]
chataf_vowels = vowel[1:4]
short_vowels = [vowel[4],vowel[6],vowel[7],vowel[11],vowel[20]]
long_vowels = ['וֹ','וּ', vowel[5], vowel[8], aleph_bet[9], 'ֹ']
vowels_limited = vowel[0:4]+short_vowels+long_vowels
dagesh = 'ּ'
maqqaf = '־'
meteg = 'ֽ'
dot = '֯'
silluq = '׃'
colon = ':'
other_char = ['ׂ','ׁ','ֿ','֯','ׅ','']
all_char = aleph_bet+cant+vowel+other_char

gutterals = ['א','ה','ח','ע','ר' ,'ﬡ','ﬣ','ﬧ']             

In [2]:
letters_w_final = ['כ','מ','נ','פ','צ']
final_letters = ['ך','ם','ן','ף','ץ']

In [3]:
shem = 'יהוה'
shem_vowels = 'יְהֹוָה'
yy_vowels = aleph_bet[9]+vowel[0]+aleph_bet[9]+vowel[8]

kal_backwards_dagesh = 'כָּל'
kal_dagesh = 'כָּל'
kal = aleph_bet[52] + vowel[8] + aleph_bet[12]
khal = aleph_bet[10] + vowel[8] + aleph_bet[12]
kol = aleph_bet[52] + vowel[20] + aleph_bet[12]
kol_maqqaf = aleph_bet[52] + vowel[20] + aleph_bet[12] + '־'
khol_maqqaf = aleph_bet[10] + vowel[20] + aleph_bet[12] + '־'
kol_space = aleph_bet[52] + vowel[20] + aleph_bet[12] + ' '
khol_space = aleph_bet[10] + vowel[20] + aleph_bet[12] + ' '

et = 'אֶת'
ve_et = 'וְאֶת'
space_et_space = ' ' + 'אֶת' + ' '
space_ve_et_space = ' ' + 'וְאֶת' + ' '
et_maqqaf = ' ' + 'אֶת' + '־'
ve_et_maqqaf = ' ' + 'וְאֶת' + '־'

In [4]:
shva_exceptions = ['שְׁתֵּי','שְׁתָּיִם','שְׁתַּיִם','שְׁנִַים','שְׁנֵי','שְׁתֵּים','שְׁנֵים','שְׁנֵי','שְׁנֵים','שְׁנָיִם']
battim = 'בָּתִּים'
batteinu = 'בָתֵּֽינוּ'
vattim = 'בָתִּים'
battei = 'בָּתֵּי'
ana = 'אָנָּא'
ana2 = 'אָנָּא'
ana3 = 'אָנָּא'
anah = 'אָנָּה'
kamatz_katan_exceptions = [battim, vattim, ana, ana2, ana3, anah, batteinu]
known_nasog_achor = ['היתה','אזרו']
known_shva_nach = ['לילה']

In [5]:
#the shem converter "eats" some special characters next to sheimot, this stops it from doing that
#it doesn't work all that well, so often manual intervention is necessary.
special_characters = ['{', '}','\\']

## Strips vowels and cantelation marks from words

In [6]:
#removes trop and vowels
def nonalpha_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter.isalpha() == True:
            no_cant_word = no_cant_word + letter
    return no_cant_word

In [7]:
#removes trop but not vowels
def trop_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter not in cant:
            no_cant_word = no_cant_word + letter
    return no_cant_word

## Converts Shem-Havaya to double-yud while preserving cantelation marks

In [8]:
def shem_converter(word):
    prefix = ''
    if word[0] != aleph_bet[9]:
        prefix = prefix + word[0]
        i = 1
        if word[1].isalpha() == False:
            prefix = prefix + word[1]
            if word[2].isalpha() == False:
                prefix = prefix + word[2]
    no_prefix_word = word[len(prefix):]

    if len(prefix) > 0:
        yud1 = aleph_bet[9]
    else:
        yud1 = aleph_bet[9]+vowel[0]

    yud2 = aleph_bet[9]+vowel[8]

    cant_marks = []

    #finds suffix
    suffix = ''
    i = -1
    while word[i] != aleph_bet[4]:
        if word[i] in special_characters:
            suffix = word[i] + suffix
        i = i-1
    
    for character in no_prefix_word:
        if character in cant:
            cant_marks.append(character)
    if len(cant_marks) == 0:
        new_shem = prefix + yud1 + yud2 + suffix
    elif len(cant_marks) == 1:
        new_shem = prefix + yud1 + yud2 + cant_marks[0] + suffix
    elif len(cant_marks) == 2:
        new_shem = prefix + yud1 + cant_marks[0] + yud2 + cant_marks[1] + suffix

    return new_shem

In [9]:
## Creates new paragraph with double-yud in place of Shem Havaya

In [10]:
#this needs to be fixed to stop eating special characters (i.e. brackets, \)

In [11]:
def convert_shem(paragraph):
    paragraph = str.replace(paragraph, '־', ' ־ ')
    par_list = paragraph.split()
    for word in par_list:
        index = par_list.index(word)
        if shem in nonalpha_remover(word):
            double_yud = shem_converter(word)
            par_list[index] = double_yud

    new_par = ' '.join(par_list)
    new_par = str.replace(new_par, ' ־ ','־')
    new_par = str.replace(new_par, '׃', '׃')
    return new_par

In [12]:
## use https://he.wikisource.org/wiki/%D7%9E%D7%A7%D7%A8%D7%90 for testing texts

In [13]:
text = 'הוֹשִׁ֤יעָה \source{תהלים כח}אֶת־עַמֶּ֗ךָ וּבָרֵ֥ךְ אֶת־נַֽחֲלָתֶ֑ךָ וּֽרְעֵ֥ם וְ֝נַשְּׂאֵ֗ם עַד־הָֽעוֹלָֽם׃ \source{מל״א ח}וְיִֽהְי֨וּ דְבָרַ֜י אֵ֗לֶּה אֲשֶׁ֤ר הִתְחַנַּ֙נְתִּי֙ לִפְנֵ֣י יְהֹוָ֔ה קְרֹבִ֛ים אֶל־יְהֹוָ֥ה אֱלֹהֵ֖ינוּ יוֹמָ֣ם וָלָ֑יְלָה לַעֲשׂ֣וֹת ׀ מִשְׁפַּ֣ט עַבְדּ֗וֹ וּמִשְׁפַּ֛ט עַמּ֥וֹ יִשְׂרָאֵ֖ל דְּבַר־י֥וֹם בְּיוֹמֽוֹ׃ לְמַ֗עַן דַּ֚עַת כׇּל־עַמֵּ֣י הָאָ֔רֶץ כִּ֥י יְהֹוָ֖ה ה֣וּא הָאֱלֹהִ֑ים אֵ֖ין עֽוֹד׃'
convert_shem(text)

'הוֹשִׁ֤יעָה \\source{תהלים כח}אֶת־עַמֶּ֗ךָ וּבָרֵ֥ךְ אֶת־נַֽחֲלָתֶ֑ךָ וּֽרְעֵ֥ם וְ֝נַשְּׂאֵ֗ם עַד־הָֽעוֹלָֽם׃ \\source{מל״א ח}וְיִֽהְי֨וּ דְבָרַ֜י אֵ֗לֶּה אֲשֶׁ֤ר הִתְחַנַּ֙נְתִּי֙ לִפְנֵ֣י יְיָ֔ קְרֹבִ֛ים אֶל־יְיָ֥ אֱלֹהֵ֖ינוּ יוֹמָ֣ם וָלָ֑יְלָה לַעֲשׂ֣וֹת ׀ מִשְׁפַּ֣ט עַבְדּ֗וֹ וּמִשְׁפַּ֛ט עַמּ֥וֹ יִשְׂרָאֵ֖ל דְּבַר־י֥וֹם בְּיוֹמֽוֹ׃ לְמַ֗עַן דַּ֚עַת כׇּל־עַמֵּ֣י הָאָ֔רֶץ כִּ֥י יְיָ֖ ה֣וּא הָאֱלֹהִ֑ים אֵ֖ין עֽוֹד׃'

## Puts a kamatz katan and maqqaf in "kol"

In [14]:
#this script does not work on words with trope
#Since pesukim from wikisource have maqqafs and kamatz katans, it shouldn't make it less useful

In [15]:
def kamatz_exception(word):
    if word[-1] != aleph_bet[12]:
        return True
    elif aleph_bet[9] in word:
        return True
    elif aleph_bet[0] in word:
        return True
    else:
        return False
    #returns "true" if the word is a likely false positive for the word "kol"

In [16]:
def kol_kamatz_katan(paragraph):
    paragraph = str.replace(paragraph, '־', ' ־ ')
    par_list = paragraph.split()
    #Paragraph is now split into words
    for word in par_list:
        index = par_list.index(word)
        if (kal in word or khal in word or kal_backwards_dagesh in word or kal_dagesh in word) and kamatz_exception(word) == False:
            if kal_backwards_dagesh in word:
                word = word.replace(kal_backwards_dagesh,kol)
            if kal_dagesh in word:
                word = word.replace(kal_dagesh,kol)
            split_word = word.split(vowel[8])
            word = vowel[20].join(split_word)
            par_list[index] = word
            #If "k(h)al" appears in a word, change it to a kamatz katan
    new_par = ' '.join(par_list)
    new_par = str.replace(new_par, ' ־ ','־')
    #Put paragraph back together
    if kol_space in new_par: 
        new_par_split = new_par.split(kol_space)
        new_par = kol_maqqaf.join(new_par_split)
    if khol_space in new_par:
        new_par_split = new_par.split(khol_space)
        new_par = khol_maqqaf.join(new_par_split)
    #Previous if-statements swap a space following "kol" to a maqqaf
    new_par = str.replace(new_par, '׃', '׃')
    return new_par

## Puts a Maqqaf after "et" (when it has a segol)

In [17]:
#this script does not work on words with trope
#Since pesukim from wikisource have maqqafs, this shouldn't be a problem.

In [18]:
def et_fixer(paragraph):
    par_list = paragraph.split()
    if et in par_list:
        new_par_split = paragraph.split(space_et_space)
        new_paragraph = et_maqqaf.join(new_par_split)
        new_par_split = new_paragraph.split(space_ve_et_space)
        new_paragraph = ve_et_maqqaf.join(new_par_split)
    else:
        new_paragraph = paragraph
    return new_paragraph

## Puts a kamatz katan in common kamatz-katan words and situations

In [19]:
#this script does not work on words with trope
#Since pesukim from wikisource have kamatz katans, this shouldn't be too much of a problem.

In [20]:
#removes shva marks, to check for kamatz katans
#Since some may preceed erroneously marked shvas
#This will need to be adjusted if the shva na marker is not a rafe
def shva_mark_remover(word):
    if rafe in word:
        word = word.replace(rafe,'')
    if dot in word:
        word = word.replace(dot,'')
    return word

In [21]:
#this imports a list of kamatz-katan words to fix and their replacements
#import csv
import csv
words_to_fix = []
corrected_words = []
with open('kamatz_correction_list.csv', 'r', encoding='utf-8') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        words_to_fix.append(shva_mark_remover(row[0]))
        corrected_words.append(row[1])
words_to_fix=words_to_fix[1:]
corrected_words=corrected_words[1:]

In [22]:
def kamatz_list_tester(word):
    i=0
    while i<len(words_to_fix):
        if shva_mark_remover(word)==words_to_fix[i]:
            word = corrected_words[i]
        i=i+1
    return word

In [23]:
def kamatz_katan_dagesh_tester(word,index,i):
    next_consonant = word[i]
    if next_consonant == 'ה':
        return False
    elif word[index+1] == meteg:
        return False
    elif next_consonant in letter_with_dagesh:
        return True
    elif word[i+1] == dagesh:
        return True
    else:
        return False

In [24]:
def kamatz_katan_adder(word):    
    #prevents script from erroneously fixing false positives
    nt_word = nonalpha_remover(word)
    for index in range(0,len(kamatz_katan_exceptions)):
        if nonalpha_remover(kamatz_katan_exceptions[index]) in nt_word:
            return word
    
    #runs word through list of predefined kamatz words to check
    if shva_mark_remover(word) in words_to_fix:
        return kamatz_list_tester(word)
    elif (shva_mark_remover(word)+colon or shva_mark_remover(word)+silluq) in words_to_fix:
        return kamatz_list_tester(word)+silluq
    
    for index in range(1,len(word)-3):
        if word[index] == vowel[8]:
            i = index
            while word[i] not in aleph_bet:
                i = i+1
                if i>len(word)-1:
                    break
            if i<len(word)-1:
                next_consonant = word[i]
                if kamatz_katan_dagesh_tester(word,index,i)==True:
                    word = word[:index]+vowel[20]+word[index+1:]
            
            i = index+1
            while word[i] not in vowels_limited:
                i = i+1
                if i>len(word)-1:
                    return word
            next_vowel = word[i]
            if next_vowel == chataf_vowels[2] and next_consonant in gutterals:
                word = word[:index]+vowel[20]+word[index+1:]
                return word
            #finds the vowel after the next one.  If both are shvas, the kamatz is katan.
            if next_vowel == shva and shva in word[index+1:]:
                if i<len(word)-1:
                    i = i+1
                    while i<len(word)-2 and word[i] not in vowels_limited:
                        i = i+1
                    sec_vowel = word[i]
                    if sec_vowel == shva:
                        word = word[:index]+vowel[20]+word[index:]
    return word

In [25]:
def kamatz_katan(paragraph):
    #This goes through the list of common words with kamatz katan (besides "kol")
    #and corrects them if they are present.
    #This list can be added to as needed.
    par_list = paragraph.split()
    
    #Calls the kamatz_katan_adder for each word in paragraph, if that word has a kamatz at all
    for index in range(len(par_list)):
        if vowel[8] in par_list[index]:
            par_list[index] = kamatz_katan_adder(par_list[index])
    paragraph = ' '.join(par_list)
    return paragraph  

## Swaps chataf-patach to shva

In [26]:
#this changes chataf-patach to shva when there's different conventions to when chataf-patach.
#this script finds chataf-patach not under gutteral letters, and changes it to a shva
def chataf_shva_swapper(word):
    chataf_patach = 'ֲ'
    if chataf_patach in word:
        for index in range(1,len(word)-3):
            i = index
            if word[index] != chataf_patach:
                continue

            while word[i] not in aleph_bet:
                i = i-1
            previous_consonant = word[i]
            if previous_consonant in gutterals[0:4]+gutterals[6:7]:
                continue
            else:
                word = word[:index]+shva+word[index+1:]
    return word

In [27]:
#Calls the shva_na_function for each word in the input paragraph
def chataf_converter(paragraph):
    par_list = paragraph.split()
    for word in par_list:
        index = par_list.index(word)
        par_list[index] = chataf_shva_swapper(word)
    new_par = ' '.join(par_list)
    return new_par

## Marks shva na'

In [28]:
#defines how shva is marked
mark = rafe
#mark = dot

In [29]:
#Inserts the marker to mark a shva na'
def na_marker(paragraph,index):
    if paragraph[index-1] == mark or paragraph[index+1] == mark or paragraph[index+2] == mark or paragraph[index-2]==mark:
        return paragraph
        #does nothing, if the shva is already marked with a rafe
    else:
        par_start = paragraph[:index]
        par_end = paragraph[index:]
        new_par = par_start+mark+par_end
        return new_par
        #adds a rafe over the input letter

In [30]:
def long_vowel_checker(word,i,index):
    previous_vowel = word[i]
    while word[i] not in aleph_bet:
        i = i-1
    word_sec = word[i:index]
    #if there's no trop and the previous syllable has a meteg, it's a long vowel with primary stress and the shva is nach
    if any(trop in cant2 for trop in word)==False and meteg in word_sec:
        return False
    trop_count = 0
    for char in word:
        if char in cant2:
            trop_count = trop_count+1
    #if there's 2 trops and 1 is on the previous syllable, the 2nd will be on the last syllable
    #therefore the 1st will be a long vowel with secondary stress, and the shva will be na
    #and this second trope will be on the last syllable, so the shva following a syllable with a trop will be the first one
    #and stress will be secondary, so the shva will be na
    if trop_count == 2 and any(trop in cant2 for trop in word_sec)==True:
        return True
    if trop_count == 2 and any(trop in cant2 for trop in word_sec)==False:
        return True
    #if there's one trop on the word and it's not on the previous syllable, it's unstressed, and the shva is na
    elif trop_count == 1 and any(trop in cant2 for trop in word_sec)==False:
        return True
    #if there's a trop on the previous syllable, assume it's stressed and nach, but it could be nasog achor and na
    #therefore these will be printed for manual checking
    elif trop_count == 1 and any(trop in cant2 for trop in word_sec)==True:
        bare_word = nonalpha_remover(word)
        for element in known_nasog_achor:
            if element in bare_word:
                return True
        for element in known_shva_nach:
            if element in bare_word:
                return True
        print('Check shva in',word,'for nasog achor')
        return False
    elif trop_count == 0 and meteg not in word_sec:
        return True
    else:
        print('Check shva in',word)

In [31]:
#Determines what shvas are na' in a word, and calls the program to mark them
def shva_na_function(word):
    nt_word = trop_remover(word)
    #this skips instances of shem hashem
    if shem_vowels in nt_word or yy_vowels in nt_word:
        return word
    for excep in shva_exceptions:
        if excep in nt_word:
            return word
    if shva not in word:
        return word
    start = 1
    ha_meteg = 'הַֽ'
    shuruk_meteg = 'וּֽ'
    if word[0:3] == ha_meteg or word[0:3] == shuruk_meteg:
        shva_loc = word.find(shva)
        word = na_marker(word,shva_loc)
        start = shva_loc+1
    
    for index in range(start,len(word)-2):
        i = index
        if word[index] != shva:
            continue
            #this doesn't bother with the loop if the letter isn't a shva
        if word[i+1] not in all_char:
            break
        while word[i] not in aleph_bet or word[i]== dagesh:
            i = i-1
        previous_consonant = word[i]
        #if the previous consonant is the beginning of the word, the shva is na'
        if i == 0:
            word = na_marker(word,index)
            continue
        if word[i-1] not in all_char:
            word = na_marker(word,index)
            continue
        
        if word[i-1] == maqqaf:
            word = na_marker(word,index)
        #if the previous consonant has a dagesh, the shva must be na'
        if previous_consonant in letter_with_dagesh:
            word = na_marker(word,index)
            continue
        if word[i+1] == dagesh or word[index+1] == dagesh:
            word = na_marker(word,index)
            continue
            
        #determines the next consonant
        i = index + 1
        while word[i] not in aleph_bet and i<len(word)-1:
            i = i+1
        next_consonant = word[i]
        if next_consonant == previous_consonant:
            word = na_marker(word,index)
            continue
        i=0
        while i<5:
            if word[index]==letters_w_final[i] and next_consonant == final_letter[i]:
                word = na_marker(word,index)
                continue   
            i=i+1

        #determines the previous vowel
        i = index-2
        while word[i] not in vowels_limited and i>0:
            i = i-1
        if i == 0:
            continue
        else:
            previous_vowel = word[i]
            #if the previous vowel is a shva, the current shva is na'
            if previous_vowel == shva:
                word = na_marker(word,index)
                continue
            if previous_vowel in long_vowels and long_vowel_checker(word,i,index)==True:
                word = na_marker(word,index)   
        
        #a shva following a long vowel is na', unless the long vowel is word-initial shuruk
   

    #figure out plan for סקינמלוי letters after הַ־ without dagesh
    #figure out plan for letters following shuruk with meteg
    #for now, neither of these get a shva na
    return word

In [32]:
#Calls the shva_na_function for each word in the input paragraph
def shva_na_converter(paragraph):
    par_list = paragraph.split()
    for word in par_list:
        index = par_list.index(word)
        par_list[index] = shva_na_function(word)
    new_par = ' '.join(par_list)
    return new_par

In [33]:
test = 'אֵ֖ת יְיָ֣ אֱלֹהֶ֑יךָ בְּכׇל־לְבָבְךָ֥ וּבְכׇל־נַפְשְׁךָ֖ וּבְכׇל־מְאֹדֶֽךָ׃'
shva_na_converter(test)

'אֵ֖ת יְיָ֣ אֱלֹהֶ֑יךָ בְּֿכׇל־לְֿבָבְךָ֥ וּבְכׇל־נַפְשְֿׁךָ֖ וּבְכׇל־מְֿאֹדֶֽךָ׃'

## Changes shva marking character

In [34]:
#this program allows the changing of all rafes used to mark shva into another character
#in this case a circle above the letter
#this could be adapted to change a text that uses the dot to using something else
def shva_na_mark_changer(paragraph):
    paragraph = paragraph.replace(rafe,'')
    paragraph = paragraph.replace(dot,'')
    return paragraph
    #return new_paragraph

## Changes colons to silluk

In [35]:
def colon_changer(line):
    #this avoids changing colons in instructions to silluk, as these should remain colons
    if 'instruction' in line:
        return line
    else:
        return line.replace(colon,silluq)

## Fixes mistakes

In [36]:
def mistake_fixer(paragraph):
    badkamatz1 = vowel[8] + vowel[20]
    badkamatz2 = vowel[20] + vowel[8]
    badkamatz3 = vowel[20]+vowel[20]
    
    baddot = dot+dot
    baddot2 = dot+shva+dagesh+dot
    baddot3 = 'ְּׁ֯֯'
    fixdot3 = baddot3[1:6]
    baddot4 = 'ְׁ֯֯'
    fixdot4 = 'ְׁ֯'
    
    badrafe = rafe+rafe
    badrafe2 = rafe+shva+dagesh+rafe
    badrafe3 = baddot3.replace(rafe,dot)
    fixrafe3 = fixdot3.replace(rafe,dot)
    badrafe4 = baddot4.replace(rafe,dot)
    fixrafe4 = fixdot4.replace(rafe,dot)
    
    if vowel[5] in paragraph or vowel[2] in paragraph:
        paragraph = paragraph.replace(badkamatz1,vowel[20])
        paragraph = paragraph.replace(badkamatz2,vowel[20])
        paragraph = paragraph.replace(badkamatz3,vowel[20])
    
    if dot in paragraph:
        paragraph = paragraph.replace(baddot,dot)
        paragraph = paragraph.replace(baddot2,dagesh+shva+dot)
        paragraph = paragraph.replace(baddot3,fixdot3)
        paragraph = paragraph.replace(baddot4,fixdot4)
    
    if rafe in paragraph:
        paragraph = paragraph.replace(badrafe,rafe)   
        paragraph = paragraph.replace(badrafe2,dagesh+shva+dot)
        paragraph = paragraph.replace(badrafe4,fixrafe4)
        
    return paragraph

## Runs Paragraph through all converters

In [37]:
def text_fixer(paragraph):
    #Comment out components of the script you don't want to run
    paragraph = shva_mark_remover(paragraph)
    #functions that only are needed for what's not copied from MAPM:
    if any(trop in cant2 for trop in paragraph)==False:
        paragraph = kol_kamatz_katan(paragraph)
        paragraph = et_fixer(paragraph)
        paragraph = kamatz_katan(paragraph)
        paragraph = chataf_converter(paragraph)
    paragraph = mistake_fixer(paragraph)
    return paragraph

In [38]:
def text_converter(paragraph):
    #Comment out components of the script you don't want to run
    #paragraph = text_fixer(paragraph)
    #paragraph = convert_shem(paragraph) 
    paragraph = shva_na_converter(paragraph)
    #paragraph = colon_changer(paragraph)
    #paragraph = shva_na_mark_changer(paragraph
    return paragraph

In [44]:
test = 'לַֽיהֹוָ֥ה הַיְשׁוּעָ֑ה עַֽל־עַמְּךָ֖ בִרְכָתֶ֣ךָ סֶּֽלָה׃ '
convert_shem(test)

'לַֽייָ֥ הַיְשׁוּעָ֑ה עַֽל־עַמְּךָ֖ בִרְכָתֶ֣ךָ סֶּֽלָה׃'


## Missing bracket finder

In [39]:
def bracket_counter(line):
    open_brace_count = line.count('{')
    close_brace_count = line.count('}')
    output = [open_brace_count,close_brace_count]
    return output

## Kamatz-check list

In [40]:
known_kamatz_gadol = ['ברכ','עלמ','קרבינו','היתה','בּרכ','דעסקין','משרתיו','ואמרו','ואמרין','הלכו']

In [41]:
#This outputs a list of lines and words that have ambiguous kamatz cases
#These are words with a kamatz that isn't marked katan followed by a shva
#Once the list is output the user can manually go through the list, find words that need a kamatz katan, and make the change
def kamatz_inspector(word):
    i=1 #since a word can't start with a kamatz
    length = len(word)
    nv_word = nonalpha_remover(word)
    #skips words with cantillation, since they'll get a kamatz katan with verse re-copying
    for stem in known_kamatz_gadol:
        if stem in nv_word:
            return False

    while i<len(word)-6:
        if word[i]==vowel[8]:
            index=i+2
            while word[index] not in vowels_limited and index<len(word)-2:
                index=index+1
            next_vowel=word[index]
            if next_vowel==shva and word[i+1] != meteg and word[index-1] not in final_letters:
                return True
        i=i+1
    return False       

In [42]:
def kamatz_finder(line,number):
    par_list = line.split()
    output = []
    #skips words with trop, since pesukim will get their kamatzes re-copied anyway
    if any(trop in cant2 for trop in line):
        return
    for word in par_list:
        if shva in word and 'ָ' in word and kamatz_inspector(word)==True:
            output.append(shva_mark_remover(word))
    if output != []:
        print(output,'in line', number)  

## Reading file from disk

In [42]:
input_file = 'siddur.tex'

In [43]:
with open(input_file, 'r', encoding='utf-8') as infile:
    lines = list(infile.readlines())

In [44]:
new_lines = []
line_number = 1
for line in lines:
    new_line = text_converter(line)
    new_lines.append(new_line)
    #for troubleshooting--identifies the last line where the script ran
    #print('line number', line_number, 'outputted successfully')
    
    
    if bracket_counter(line) != bracket_counter(new_line):
        print('Check brackets in line ',line_number)
    
    kamatz_finder(new_line,line_number) 
    
    line_number=line_number+1
    

Check shva in יִשָּׂא֑וּנְךָ for nasog achor
['מְעָרְבִין', 'אָמְרָה:'] in line 417
Check shva in וֶ֝אֱמ֥וּנָתְךָ֗
Check shva in וְֿכׇל־שֹׁ֥פְטֵי for nasog achor
Check shva in גָּאָ֑לְתָּ for nasog achor
['הַמְבֹרָךְ׃}\\end{Large}'] in line 1036
['מְשָׁרְתִים', 'מְשָׁרְתָיו'] in line 1067
['וְאָמְרוּ׃'] in line 1173
['וְקָרְבֵֽנוּ'] in line 1276
['כְּשֶׁעָמְדָה', 'הָרְשָׁעָה'] in line 1425
['אָבְדָה'] in line 1741
['הַמְבֹרָךְ׃}\\\\'] in line 1899
Check shva in וְֿיֹ֣שְׁבֵי for nasog achor
Check shva in כׇּל־יֹ֥שְׁבֵי for nasog achor
['כְּשֶׁעָמְדָה', 'הָרְשָׁעָה'] in line 2494
['כׇּל־צָרְכָּנָא,'] in line 2742
Check shva in כׇּל־עֹ֬בְדֵי for nasog achor
Check shva in וְֿיֹ֣שְׁבֵי for nasog achor
['וּמְעָרְבִין'] in line 2946
['כְּשֶׁעָמְדָה', 'הָרְשָׁעָה'] in line 3125
Check shva in דָּ֭רְשָׁה for nasog achor
Check shva in נָ֣טְעָה for nasog achor
Check shva in תָּ֣מְכוּ for nasog achor
Check shva in עָ֭שְׂתָה for nasog achor
Check shva in לִמְנוּחָ֑יְכִי for nasog achor
Check shva in וֶ֝אֱמ֥וּנָתְך

In [45]:
output_file = 'siddur_script.tex'

In [46]:
with open(output_file, 'w', encoding='utf-8') as outfile:
    for line in new_lines:
        if line != '':
            if line[-1] != '\n':
                outfile.write(line + '\n')
            else:
                outfile.write(line)
        else:
            outfile.write('\n')